In [3]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

import sys
sys.path.append('/content/gdrive/My Drive/workspace')

%cd 'gdrive/My Drive/workspace'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import cv2

In [0]:
import LBP
import preprocessing as prep

import numpy as np
from collections import Counter
import matplotlib.pyplot as plt


import dlib

from PIL import Image


from importlib import reload

_ = reload(prep)

from sklearn.model_selection import train_test_split


import helpers
from os.path import join
reload(helpers)
import math

import pandas as pd

In [0]:
predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector_path = 'mmod_human_face_detector.dat'
root_path = 'lfw'
people_txt = 'people.txt'

# predictor_path = 'gdrive/My Drive/frworkspace/shape_predictor_68_face_landmarks.dat'
# detector_path = 'gdrive/My Drive/frworkspace/mmod_human_face_detector.dat'
# root_path = 'gdrive/My Drive/lfw/lfw'
# people_txt = 'gdrive/My Drive/lfw/people.txt'

In [0]:

def get_image(path):
    img = dlib.load_rgb_image(path)
    funneled_image_arr = prep.cropped_and_aligned(img, cnn_face_detector, sp,method='cnn',size=150)
    fun_image = Image.fromarray(funneled_image_arr).convert('L')
    gray_arr = np.asarray(fun_image)
    return gray_arr


In [0]:
cnn_face_detector = dlib.cnn_face_detection_model_v1(detector_path)

sp = dlib.shape_predictor(predictor_path)

data load start here

In [0]:
df = helpers.load_names(people_txt, 6,20)
names = set(df['name'])
df['image_path'] = df.apply(lambda row: join(root_path, row['name'], row['name']+'_'+str(row['image_num']).zfill(4) + '.jpg'),axis=1)

In [15]:
df.shape, len(names)

((2502, 3), 254)

In [0]:
df['label'] = pd.Categorical(df['name'])
df['label'] = df.label.cat.codes

rest, test = train_test_split(df, test_size=0.2, stratify=df['name'])
train, _  = train_test_split(rest, test_size=0.2, stratify=rest['name'])

In [17]:
print(test.shape, train.shape), len(set(train.name)), len(set(test.name)), len(set(train.label)), len(set(test.label))

(501, 4) (1600, 4)


(None, 254, 254, 254, 254)

In [18]:
faces = (train.image_path.apply(get_image)).values
labels = np.array((train.label).values,dtype=int)

/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 2 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))
/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 3 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))


In [19]:
test['face'] = test.image_path.apply(get_image)

/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 2 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))
/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 3 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
model = cv2.face.LBPHFaceRecognizer_create(radius=9)


model.train(faces, labels)

In [21]:
p = 0
n = 0
no = 1
for row in test.itertuples():
    truth = row.label
    test_face = row.face
    predicted, confidence = model.predict(test_face)
    
    if truth == predicted:
        p+=1
    else:
        n+=1
    if no%100 == 0: 
        print(no, p/(p+n))
    no+=1
print(p,n, p/(p+n))

100 0.5
200 0.52
300 0.51
400 0.5075
500 0.488
244 257 0.4870259481037924


In [0]:
# 190 311 0.37924151696606784 # radius1
# 208 293 0.4151696606786427 # radius2
# 217 284 0.43313373253493015 # radius3
# 240 261 0.47904191616766467 # radius4
# 249 252 0.49700598802395207 # r5
# 257 244 0.5129740518962076 # r6
# 243 258 0.48502994011976047 # r7
# 244 257 0.4870259481037924 # R8
# 235 266 0.469061876247505 # 10
